<a href="https://colab.research.google.com/github/AmmarAhmedl200961/twitter-sentiment-analysis-pyspark/blob/main/l200961.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task#01:

## Data Gathering:

Setting up HDFS

In [ ]:
# Install jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz
!tar xzvf hadoop-3.2.4.tar.gz

# !wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
# !tar xf spark-3.1.2-bin-hadoop3.2.tgz

Streaming output truncated to the last 5000 lines.
hadoop-3.2.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-common/apidocs/org/apache/hadoop/yarn/server/records/package-summary.html
hadoop-3.2.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-common/apidocs/org/apache/hadoop/yarn/server/api/
hadoop-3.2.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-common/apidocs/org/apache/hadoop/yarn/server/api/CollectorNodemanagerProtocol.html
hadoop-3.2.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-common/apidocs/org/apache/hadoop/yarn/server/api/class-use/
hadoop-3.2.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-common/apidocs/org/apache/hadoop/yarn/server/api/class-use/CollectorNodemanagerProtocol.html
hadoop-3.2.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-common/apidocs/org/apache/hadoop/yarn/server/api/class-use/SCMUploaderProtocol.html
hadoop-3.2.4/share/doc/ha

In [ ]:
import os
# Set jdk environment path which enables you to run Pyspark in your Colab environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
os.environ["HADOOP_HOME"] = "/content/hadoop-3.2.4"


update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [ ]:
!/content/hadoop-3.2.4/bin/hadoop version

Hadoop 3.2.4
Source code repository Unknown -r 7e5d9983b388e372fe640f21f048f2f2ae6e9eba
Compiled by ubuntu on 2022-07-12T11:58Z
Compiled with protoc 2.5.0
From source with checksum ee031c16fe785bbb35252c749418712
This command was run using /content/hadoop-3.2.4/share/hadoop/common/hadoop-common-3.2.4.jar


Configuring kaggle dataset

In [ ]:
!pip install opendatasets

import opendatasets as od
od.download('https://www.kaggle.com/datasets/kazanova/sentiment140')

100%|██████████| 80.9M/80.9M [00:00<00:00, 110MB/s] 


Upload to HDFS

In [ ]:
# Upload to hdfs the file we downloaded

!/content/hadoop-3.2.4/bin/hadoop fs -mkdir /sentiment140
!/content/hadoop-3.2.4/bin/hadoop fs -put sentiment140/training.1600000.processed.noemoticon.csv /sentiment140


In [ ]:
# Check hadoop filesystem

!/content/hadoop-3.2.4/bin/hadoop fs -ls /sentiment140


Found 1 items
-rw-r--r--   1 root root  238803811 2024-04-20 17:52 /sentiment140/training.1600000.processed.noemoticon.csv


In [ ]:
!pip install -q findspark pyspark==3.3.0 spark-nlp==4.2.8 kaggle nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.8 MB/s eta 0:00:00


In [ ]:
# import findspark
# findspark.init('spark-3.1.2-bin-hadoop3.2')

In [ ]:
# from pyspark.sql.session import SparkSession
# spark = SparkSession.builder \
#     .master('local[*]')\
#     .config('spark.driver.memory','16G')\
#     .config('spark.driver.maxResultSize', '0') \
#     .config('spark.kryoserializer.buffer.max', '2000M')\
#     .config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.1')\
#     .getOrCreate()
import sparknlp
spark = sparknlp.start()
spark

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
import os

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, to_timestamp, expr
from pyspark.sql.functions import udf
from pyspark.sql import functions as F

# adding columns to dataset


In [ ]:
# Load the data from HDFS
df =(spark.read
          .format('csv')
          .option('header', 'false')
          .load('/sentiment140/training.1600000.processed.noemoticon.csv'))
# Show the DataFrame
df.show(truncate=False)

+---+----------+----------------------------+--------+---------------+---------------------------------------------------------------------------------------------------------------------+
|_c0|_c1       |_c2                         |_c3     |_c4            |_c5                                                                                                                  |
+---+----------+----------------------------+--------+---------------+---------------------------------------------------------------------------------------------------------------------+
|0  |1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  |
|0  |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY|scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!      |
|0  |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY|m

In [ ]:
df = (df.withColumnRenamed('_c0','target')
        .withColumnRenamed('_c1','id')
        .withColumnRenamed('_c2','tweet_date')
        .withColumnRenamed('_c3','flag')
        .withColumnRenamed('_c4','user')
        .withColumnRenamed('_c5','text')
)

df = df.withColumn('tweet_date', expr('substring(tweet_date, 5, 27)'))


df = df.select(col('target').cast('int'),
                         col('id').cast('int'),
                         to_timestamp(col('tweet_date'),'MMM dd HH:mm:ss zzz yyyy').alias('date'),
                         col('flag').cast('string'),
                         col('user').cast('string'),
                         col('text').cast('string'),
                        )
df = df.withColumn('weekday', F.date_format('date', 'EEEE'))
df.show()

+------+----------+-------------------+--------+---------------+--------------------+-------+
|target|        id|               date|    flag|           user|                text|weekday|
+------+----------+-------------------+--------+---------------+--------------------+-------+
|     0|1467810369|2009-04-07 05:19:45|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|Tuesday|
|     0|1467810672|2009-04-07 05:19:49|NO_QUERY|  scotthamilton|is upset that he ...|Tuesday|
|     0|1467810917|2009-04-07 05:19:53|NO_QUERY|       mattycus|@Kenichan I dived...|Tuesday|
|     0|1467811184|2009-04-07 05:19:57|NO_QUERY|        ElleCTF|my whole body fee...|Tuesday|
|     0|1467811193|2009-04-07 05:19:57|NO_QUERY|         Karoli|@nationwideclass ...|Tuesday|
|     0|1467811372|2009-04-07 05:20:00|NO_QUERY|       joy_wolf|@Kwesidei not the...|Tuesday|
|     0|1467811592|2009-04-07 05:20:03|NO_QUERY|        mybirch|         Need a hug |Tuesday|
|     0|1467811594|2009-04-07 05:20:03|NO_QUERY|           c

The data was created using the Twitter Search API. It has 1.6m rows and 6 columns.

Columns info by order:

• target (0 = negative, 2 = neutral,4 = positive)<br>
• ID<br>
• Tweet_date<br>
• query (NO_QUERY)<br>
• user<br>
• text (tweet content)

Official site of the dataset: http://help.sentiment140.com/for-students/

Kaggle site of the dataset: https://www.kaggle.com/kazanova/sentiment140

In [ ]:
df.printSchema()

root
 |-- target: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)
 |-- weekday: string (nullable = true)



Preprocessing Dataset

In [ ]:
df = df.dropna()
df.count(), len(df.columns)

(1167087, 7)

In [ ]:
from pyspark.sql import functions as F
def preprocessing(sparkDF,col):
    # Remove any URLs. matches 'http+'
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, r'http\S+', ''))
    # Remove any mentions (e.g., @username). matches '@+'
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, '@\w+', ''))
    # Remove any retweets
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, 'RT', ''))
    # remove any hashtags
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, '#\w+', ''))
    # Remove any emojis like :lol: etc
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, ':', ''))
    # Remove any non-alphanumeric characters
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, '[^A-Za-z0-9]+', ' '))
    # Remove any hyphens `-`
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, '\-', ''))
    # Fix sequence of spaces to a single space
    sparkDF = sparkDF.withColumn(col, F.regexp_replace(col, '[ ]+', ' '))
    # Trim leading or trailing spaces
    sparkDF = sparkDF.withColumn(col, F.trim(sparkDF[col]))

    return sparkDF

# df = preprocessing(df,'text')

# process date column

# df.show(25,truncate=False)

In [ ]:
# check duplicates
duplicates = df.groupBy("text").count().where("`count` > 1")

# Show duplicates
duplicates.show()

+--------------------+-----+
|                text|count|
+--------------------+-----+
|watching my favor...|    3|
|I d like to use F...|    2|
|is about to go to...|    3|
|Wow we have a fam...|    2|
|     Is really tired|    2|
|    i miss your face|    2|
|Starting to feel ...|    2|
|        are you okay|    5|
|did great at the ...|    6|
|I ve already been...|    2|
|     wants chocolate|    3|
|        One more day|    2|
|Monday again already|    2|
|while NOT realisi...|    2|
|Someone just call...|    2|
|    Not feeling good|    6|
|Food was great as...|    2|
|aww why dont you ...|    2|
|I know I just cau...|    2|
|eating some doubl...|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# drop duplicates
df = df.dropDuplicates(['text'])
df.groupBy("text").count().where("`count` > 1").show()

+----+-----+
|text|count|
+----+-----+
+----+-----+



In [ ]:
# Display users who tweeted most
df.groupBy('user').count().sort('count',ascending=False).show(10)

+--------------+-----+
|          user|count|
+--------------+-----+
|       webwoke|  341|
|   mcraddictal|  258|
|      tsarnick|  247|
|   VioletsCRUK|  228|
|        keza34|  217|
|     DarkPiano|  213|
|  thisgoeshere|  204|
|ramdomthoughts|  200|
|     shanajaca|  194|
|       Dogbook|  192|
+--------------+-----+
only showing top 10 rows



In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('english')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# decorator for udf
@udf(returnType=StringType())
def clean_tweet(tweet):

  word_tokens = word_tokenize(tweet)

  filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words and w.isalpha()]

  return ' '.join(filtered_sentence)

df = df.withColumn('clean_text', clean_tweet(col('text')))

df.show()

+------+----------+-------------------+--------+---------------+--------------------+---------+--------------------+
|target|        id|               date|    flag|           user|                text|  weekday|          clean_text|
+------+----------+-------------------+--------+---------------+--------------------+---------+--------------------+
|     0|1880891315|2009-05-22 09:52:19|NO_QUERY|           besz|0 0 I saw you on ...|   Friday|saw screen tonigh...|
|     0|1986546708|2009-06-01 01:59:12|NO_QUERY|         MarceP|0 0 and the kiss ...|   Monday|kiss want see hap...|
|     0|1676253106|2009-05-02 04:30:38|NO_QUERY|           besz|     0 0 where is it| Saturday|                    |
|     0|1970505254|2009-05-30 11:08:12|NO_QUERY|        gaylady|     0 4 now feck it| Saturday|                feck|
|     0|2052749375|2009-06-06 08:52:31|NO_QUERY|Hotpinkstardrop|0 5lb on but I ca...| Saturday|                    |
|     4|2071639205|2009-06-08 02:42:23|NO_QUERY|       bowlwiki|

In [ ]:
# prepare dataframe for the model
df_preprocessed = df.select(col('clean_text'), col('target'))

df_preprocessed.show()

+--------------------+------+
|          clean_text|target|
+--------------------+------+
| aaaaand nausea back|     0|
|throat closing st...|     0|
|           love buck|     0|
|behind video game...|     0|
|problem tongue ch...|     0|
|wtf kutner oh pis...|     0|
|tweeting much ton...|     0|
|missed hills catc...|     0|
|new twitter name ...|     0|
|       good gone bed|     0|
|nothing sick day ...|     0|
|  threw bathroom fun|     0|
|got back funeral ...|     0|
|     ive txt morning|     0|
|choices limited s...|     0|
|okie dokie tweopl...|     0|
|         sleep dunno|     0|
|day busy head hur...|     0|
|annoying place ha...|     0|
|             problem|     0|
+--------------------+------+
only showing top 20 rows



Label the tweets with sentiment scores (positive, negative, neutral) using pre-trained
sentiment lexicons or machine learning models.

# **Constructing pretrained Model**

In [ ]:
# Import the Spark NLP library
import sparknlp
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

# Create DocumentAssembler transformer that converts text into a format for UniversalSentenceEncoder
document_assembler = DocumentAssembler().setInputCol('clean_text').setOutputCol('document')

# UniversalSentenceEncoder that uses a pretrained model from TensorFlow Hub
# It encodes text into high-density vectors that can be used for text classification and other natural language tasks
use = UniversalSentenceEncoder.pretrained(name='tfhub_use', lang='en').setInputCols(['document']).setOutputCol('sentence_embeddings')

# Create a SentimentDLModel that uses a pretrained model pretrained on twitter data
# It is a deep learning model used for sentiment analysis
sentiment_dl = SentimentDLModel.pretrained(name='sentimentdl_use_twitter', lang='en').setInputCols(['sentence_embeddings']).setOutputCol('sentiment')

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(
      stages = [
          document_assembler,
          use,
          sentiment_dl
      ])

# **Training**

In [ ]:
(training_data, test_data) = df_preprocessed.randomSplit([0.75, 0.25])


model = pipeline.fit(training_data)

In [ ]:
result = model.transform(test_data)


In [ ]:
result.show()

+----------+------+--------------------+-------------------+---------+
|clean_text|target|            document|sentence_embeddings|sentiment|
+----------+------+--------------------+-------------------+---------+
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|          |     0|[{document, 0, -1...|                 []|       []|
|     

Py4JError: An error occurred while calling o293.showString

In [ ]:
# from pyspark.sql.functions import col

# sentiment_result = result.select(
#     col("target"),
#     col("document.result").alias("document"),
#     col("sentiment.result").alias("sentiment")
# )

# sentiment_result.show(truncate=False)

In [ ]:
# same format of result but with no error
sentiment_result = result.withColumn('cols', F.explode(F.arrays_zip('document.result', 'sentiment.result'))) \
.select(F.expr("cols['0']").alias('document'), 'target',
        F.expr("cols['1']").alias('sentiment'))

sentiment_result.show(truncate=False)


+--------------------+------+
|document|target|sentiment|
+--------+------+---------+
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
|        |0     |null     |
+--------------------+------+
only showing top 20 rows



# **Model Evaluation**

In [ ]:
# the result set:  polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
RESULT_MAP = { 'positive': 4, 'negative':0, 'neutral':2 }

@udf(returnType=IntegerType())
def map_results(text):
  return RESULT_MAP[text]

sentiment_result = sentiment_result.na.drop()

final_result = sentiment_result.withColumn('result', map_results(col('sentiment')))

final_result.show()

+--------------------+------+---------+------+
|            document|target|sentiment|result|
+--------------------+------+---------+------+
|aaa migraine poo ...|     0| negative|     0|
|aaaa doooo worksh...|     0| negative|     0|
|aaaa need work to...|     0| negative|     0|
|aaaa translating ...|     0| positive|     4|
|aaaaa guua mau bl...|     0| positive|     4|
|aaaaa study stavr...|     0| negative|     0|
|aaaaaaaa hate fuc...|     0| negative|     0|
|aaaaaaaaaa day ch...|     0| negative|     0|
|   aaaaaaaaaaa mcfly|     0| positive|     4|
|aaaaaaaaaaaaaaaaa...|     0| negative|     0|
|aaaaaaaaaaaaaaaaa...|     0|  neutral|     2|
|aaaaaaaaaah still...|     0| negative|     0|
|aaaaaaaaaahhhhh t...|     0| negative|     0|
|aaaaaaaaah bs ya ...|     0| positive|     4|
|aaaaaaaaargh hote...|     0| positive|     4|
|aaaaaaaahhhhhhhhh...|     0| negative|     0|
|aaaaaaawh period ...|     0| negative|     0|
|aaaaaah ouch cort...|     0| negative|     0|
|aaaaaahhhh h

In [ ]:
predictionAndLabels = final_result.select('target', 'result')

from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix = metrics.confusionMatrix().toArray()
labels = [int(label) for label in metrics.call('labels')]
pd.DataFrame(confusion_matrix , index=labels, columns=labels)

In [ ]:
print('Summary Stats')
print(f'Accuracy: {metrics.accuracy:.4f}')
print(f'False positive rate: {metrics.weightedFalsePositiveRate:.4f}')